In [93]:
import numpy as np
import pandas as pd
from tqdm import tqdm


import catboost
from catboost import CatBoostClassifier, Pool
from catboost.eval.catboost_evaluation import *
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier, Pool, cv
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

## Load and Preprocess data

In [77]:
data = pd.read_csv('./data/features.csv')

In [78]:
data.head()

,week_day,month,year,hexagon_id,num_in,weekend,ride_distance,load_coef
0,0.0,1.0,2019.0,1.0,2296,False,12271.450007,0.621548
1,0.0,1.0,2019.0,2.0,194,False,1042.820005,0.052518
2,0.0,1.0,2019.0,3.0,392,False,1950.290004,0.106118
3,0.0,1.0,2019.0,4.0,382,False,1973.489998,0.103411
4,0.0,1.0,2019.0,5.0,1104,False,5911.430001,0.298863


In [79]:
discr_period = 0.05

data['weekend'] = data['weekend'].apply(int) 
data['load_coef'] = (data['load_coef'] // discr_period) * discr_period

In [81]:
data.columns

Index(['week_day', 'month', 'year', 'hexagon_id', 'num_in', 'weekend',
       'ride_distance', 'load_coef'],
      dtype='object')

In [95]:
le = LabelEncoder()

In [100]:
X = np.array(data[['week_day', 'month', 'year', 'hexagon_id', 'num_in', 'weekend', 'ride_distance']])
y = np.array(data['load_coef'])
y = le.fit_transform(y)


In [101]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.75, random_state=42)

/home/mak7/PycharmProjects/MURA/mur/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [105]:
model = CatBoostClassifier(
    custom_loss=['Accuracy'],
    random_seed=42,
    logging_level='Silent'
)



In [ ]:
model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    logging_level='Verbose',  # you can uncomment this for text output
#     plot=True
);



Learning rate set to 0.090498
0:	learn: 0.4752770	test: 0.4750421	best: 0.4750421 (0)	total: 83.2ms	remaining: 1m 23s
1:	learn: 0.2785065	test: 0.2779740	best: 0.2779740 (1)	total: 90.6ms	remaining: 45.2s
2:	learn: 0.1752236	test: 0.1752106	best: 0.1752106 (2)	total: 103ms	remaining: 34.1s
3:	learn: 0.1270054	test: 0.1274441	best: 0.1274441 (3)	total: 111ms	remaining: 27.7s
4:	learn: 0.0843663	test: 0.0845359	best: 0.0845359 (4)	total: 119ms	remaining: 23.7s
5:	learn: 0.0610441	test: 0.0610917	best: 0.0610917 (5)	total: 126ms	remaining: 20.8s
6:	learn: 0.0461295	test: 0.0461454	best: 0.0461454 (6)	total: 140ms	remaining: 19.9s
7:	learn: 0.0335944	test: 0.0337829	best: 0.0337829 (7)	total: 158ms	remaining: 19.6s
8:	learn: 0.0248386	test: 0.0249191	best: 0.0249191 (8)	total: 174ms	remaining: 19.2s
9:	learn: 0.0185128	test: 0.0184813	best: 0.0184813 (9)	total: 187ms	remaining: 18.5s
10:	learn: 0.0145786	test: 0.0145505	best: 0.0145505 (10)	total: 194ms	remaining: 17.4s
11:	learn: 0.011238

105:	learn: 0.0000810	test: 0.0000796	best: 0.0000796 (105)	total: 1.44s	remaining: 12.1s
106:	learn: 0.0000797	test: 0.0000783	best: 0.0000783 (106)	total: 1.46s	remaining: 12.1s
107:	learn: 0.0000788	test: 0.0000775	best: 0.0000775 (107)	total: 1.47s	remaining: 12.2s
108:	learn: 0.0000779	test: 0.0000767	best: 0.0000767 (108)	total: 1.49s	remaining: 12.1s
109:	learn: 0.0000767	test: 0.0000754	best: 0.0000754 (109)	total: 1.5s	remaining: 12.1s
110:	learn: 0.0000757	test: 0.0000745	best: 0.0000745 (110)	total: 1.51s	remaining: 12.1s
111:	learn: 0.0000745	test: 0.0000733	best: 0.0000733 (111)	total: 1.52s	remaining: 12.1s
112:	learn: 0.0000732	test: 0.0000721	best: 0.0000721 (112)	total: 1.53s	remaining: 12s
113:	learn: 0.0000722	test: 0.0000711	best: 0.0000711 (113)	total: 1.55s	remaining: 12s
114:	learn: 0.0000712	test: 0.0000700	best: 0.0000700 (114)	total: 1.56s	remaining: 12s
115:	learn: 0.0000703	test: 0.0000692	best: 0.0000692 (115)	total: 1.57s	remaining: 12s
116:	learn: 0.00006

200:	learn: 0.0000330	test: 0.0000327	best: 0.0000327 (200)	total: 2.65s	remaining: 10.5s
201:	learn: 0.0000329	test: 0.0000326	best: 0.0000326 (201)	total: 2.67s	remaining: 10.6s
202:	learn: 0.0000327	test: 0.0000323	best: 0.0000323 (202)	total: 2.69s	remaining: 10.6s
203:	learn: 0.0000324	test: 0.0000321	best: 0.0000321 (203)	total: 2.7s	remaining: 10.5s
204:	learn: 0.0000322	test: 0.0000319	best: 0.0000319 (204)	total: 2.71s	remaining: 10.5s
205:	learn: 0.0000321	test: 0.0000318	best: 0.0000318 (205)	total: 2.72s	remaining: 10.5s
206:	learn: 0.0000319	test: 0.0000316	best: 0.0000316 (206)	total: 2.73s	remaining: 10.5s
207:	learn: 0.0000317	test: 0.0000313	best: 0.0000313 (207)	total: 2.75s	remaining: 10.5s
208:	learn: 0.0000315	test: 0.0000311	best: 0.0000311 (208)	total: 2.76s	remaining: 10.4s
209:	learn: 0.0000313	test: 0.0000309	best: 0.0000309 (209)	total: 2.77s	remaining: 10.4s
210:	learn: 0.0000311	test: 0.0000307	best: 0.0000307 (210)	total: 2.78s	remaining: 10.4s
211:	learn:

300:	learn: 0.0000207	test: 0.0000206	best: 0.0000206 (300)	total: 4.09s	remaining: 9.5s
301:	learn: 0.0000206	test: 0.0000205	best: 0.0000205 (301)	total: 4.11s	remaining: 9.49s
302:	learn: 0.0000205	test: 0.0000204	best: 0.0000204 (302)	total: 4.12s	remaining: 9.47s
303:	learn: 0.0000204	test: 0.0000203	best: 0.0000203 (303)	total: 4.13s	remaining: 9.45s
304:	learn: 0.0000203	test: 0.0000203	best: 0.0000203 (304)	total: 4.14s	remaining: 9.43s
305:	learn: 0.0000203	test: 0.0000202	best: 0.0000202 (305)	total: 4.15s	remaining: 9.42s
306:	learn: 0.0000202	test: 0.0000201	best: 0.0000201 (306)	total: 4.16s	remaining: 9.4s
307:	learn: 0.0000201	test: 0.0000200	best: 0.0000200 (307)	total: 4.18s	remaining: 9.38s
308:	learn: 0.0000200	test: 0.0000199	best: 0.0000199 (308)	total: 4.19s	remaining: 9.36s
309:	learn: 0.0000199	test: 0.0000199	best: 0.0000199 (309)	total: 4.2s	remaining: 9.35s
310:	learn: 0.0000199	test: 0.0000198	best: 0.0000198 (310)	total: 4.21s	remaining: 9.33s
311:	learn: 0

394:	learn: 0.0000154	test: 0.0000154	best: 0.0000154 (394)	total: 5.35s	remaining: 8.2s
395:	learn: 0.0000154	test: 0.0000154	best: 0.0000154 (395)	total: 5.39s	remaining: 8.22s
396:	learn: 0.0000153	test: 0.0000153	best: 0.0000153 (396)	total: 5.4s	remaining: 8.2s
397:	learn: 0.0000153	test: 0.0000153	best: 0.0000153 (397)	total: 5.41s	remaining: 8.19s
398:	learn: 0.0000152	test: 0.0000152	best: 0.0000152 (398)	total: 5.42s	remaining: 8.17s
399:	learn: 0.0000152	test: 0.0000152	best: 0.0000152 (399)	total: 5.43s	remaining: 8.15s
400:	learn: 0.0000151	test: 0.0000151	best: 0.0000151 (400)	total: 5.44s	remaining: 8.13s
401:	learn: 0.0000151	test: 0.0000151	best: 0.0000151 (401)	total: 5.45s	remaining: 8.11s
402:	learn: 0.0000151	test: 0.0000151	best: 0.0000151 (402)	total: 5.47s	remaining: 8.1s
403:	learn: 0.0000150	test: 0.0000150	best: 0.0000150 (403)	total: 5.48s	remaining: 8.08s
404:	learn: 0.0000150	test: 0.0000150	best: 0.0000150 (404)	total: 5.49s	remaining: 8.06s
405:	learn: 0.

499:	learn: 0.0000119	test: 0.0000120	best: 0.0000120 (499)	total: 6.77s	remaining: 6.77s
500:	learn: 0.0000119	test: 0.0000120	best: 0.0000120 (500)	total: 6.79s	remaining: 6.76s
501:	learn: 0.0000119	test: 0.0000120	best: 0.0000120 (501)	total: 6.8s	remaining: 6.75s
502:	learn: 0.0000119	test: 0.0000119	best: 0.0000119 (502)	total: 6.81s	remaining: 6.73s
503:	learn: 0.0000119	test: 0.0000119	best: 0.0000119 (503)	total: 6.82s	remaining: 6.72s
504:	learn: 0.0000118	test: 0.0000119	best: 0.0000119 (504)	total: 6.83s	remaining: 6.7s
505:	learn: 0.0000118	test: 0.0000119	best: 0.0000119 (505)	total: 6.85s	remaining: 6.68s
506:	learn: 0.0000118	test: 0.0000118	best: 0.0000118 (506)	total: 6.86s	remaining: 6.67s
507:	learn: 0.0000117	test: 0.0000118	best: 0.0000118 (507)	total: 6.87s	remaining: 6.65s
508:	learn: 0.0000117	test: 0.0000118	best: 0.0000118 (508)	total: 6.88s	remaining: 6.64s
509:	learn: 0.0000117	test: 0.0000118	best: 0.0000118 (509)	total: 6.89s	remaining: 6.62s
510:	learn: 

591:	learn: 0.0000100	test: 0.0000101	best: 0.0000101 (591)	total: 8s	remaining: 5.52s
592:	learn: 0.0000100	test: 0.0000101	best: 0.0000101 (592)	total: 8.02s	remaining: 5.51s
593:	learn: 0.0000100	test: 0.0000101	best: 0.0000101 (593)	total: 8.04s	remaining: 5.49s
594:	learn: 0.0000100	test: 0.0000101	best: 0.0000101 (594)	total: 8.04s	remaining: 5.48s
595:	learn: 0.0000099	test: 0.0000100	best: 0.0000100 (595)	total: 8.06s	remaining: 5.46s
596:	learn: 0.0000099	test: 0.0000100	best: 0.0000100 (596)	total: 8.07s	remaining: 5.45s
597:	learn: 0.0000099	test: 0.0000100	best: 0.0000100 (597)	total: 8.08s	remaining: 5.43s
598:	learn: 0.0000099	test: 0.0000100	best: 0.0000100 (598)	total: 8.09s	remaining: 5.42s
599:	learn: 0.0000099	test: 0.0000100	best: 0.0000100 (599)	total: 8.1s	remaining: 5.4s
600:	learn: 0.0000099	test: 0.0000100	best: 0.0000100 (600)	total: 8.11s	remaining: 5.39s
601:	learn: 0.0000099	test: 0.0000100	best: 0.0000100 (601)	total: 8.13s	remaining: 5.37s
602:	learn: 0.0

692:	learn: 0.0000086	test: 0.0000087	best: 0.0000087 (692)	total: 9.46s	remaining: 4.19s
693:	learn: 0.0000086	test: 0.0000087	best: 0.0000087 (693)	total: 9.48s	remaining: 4.18s
694:	learn: 0.0000086	test: 0.0000087	best: 0.0000087 (694)	total: 9.49s	remaining: 4.17s
695:	learn: 0.0000085	test: 0.0000087	best: 0.0000087 (695)	total: 9.51s	remaining: 4.15s
696:	learn: 0.0000085	test: 0.0000086	best: 0.0000086 (696)	total: 9.52s	remaining: 4.14s
697:	learn: 0.0000085	test: 0.0000086	best: 0.0000086 (697)	total: 9.53s	remaining: 4.12s
698:	learn: 0.0000085	test: 0.0000086	best: 0.0000086 (698)	total: 9.54s	remaining: 4.11s
699:	learn: 0.0000085	test: 0.0000086	best: 0.0000086 (699)	total: 9.55s	remaining: 4.09s
700:	learn: 0.0000085	test: 0.0000086	best: 0.0000086 (700)	total: 9.56s	remaining: 4.08s
701:	learn: 0.0000085	test: 0.0000086	best: 0.0000086 (701)	total: 9.58s	remaining: 4.07s
702:	learn: 0.0000085	test: 0.0000086	best: 0.0000086 (702)	total: 9.59s	remaining: 4.05s
703:	learn